<a href="https://colab.research.google.com/github/oliveralbrecht94/ATDIT2/blob/master/Task%20E.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import tensorflow as tf
import numpy as np
import skimage as ski
import matplotlib.pyplot as plt
from scipy import stats
import scipy.spatial.distance

plt.set_cmap(None)
plt.gray()

<Figure size 432x288 with 0 Axes>

In [0]:
cifar_train = tf.keras.datasets.cifar10.load_data()[0]
train_images, train_labels = cifar_train[0], cifar_train[1]

In [0]:
cifar_test = tf.keras.datasets.cifar10.load_data()[1]
test_images, test_labels = cifar_test[0], cifar_test[1]

In [0]:
train_labels_list = []
for i in range(train_labels.shape[0]):
    train_labels_list.append(train_labels[i][0])

In [0]:
def rgb2gray(rgb):
    return np.dot(rgb[...,:3], [0.299, 0.587, 0.114])

In [0]:
y_train = train_labels
y_test = test_labels

# Images into grey

In [0]:
x_train_gray = []

for i in range(train_images.shape[0]):
    x_train_gray.append(rgb2gray(train_images[i]))
    
x_train_gray = np.array(x_train_gray)

In [0]:
x_test_gray = []

for i in range(test_images.shape[0]):
    x_test_gray.append(rgb2gray(test_images[i]))
    
x_test_gray = np.array(x_test_gray)

In [15]:
train_images.shape

(50000, 32, 32, 3)

# Preparing Dataset

In [32]:
n_image = train_images.shape[0]
n_test_image = test_images.shape[0]
m = train_images.shape[1]**2


# Transform to 1D vector from each matrix with the pixels of an image in order to calculate distances.
gray_image = x_train_gray.reshape([n_train, m])
gray_test_image = x_test_gray.reshape([n_test, m])
gray_image.shape

(50000, 1024)

In [33]:
size_image_sample = 1000
index = np.random.randint(n_train, size = size_image_sample) # 5000 [1000] zufällige Indexwerte für das MNIST-Set
image_sample = gray_image[index]
labels_sample = train_labels[index]
image_sample.shape

(1000, 1024)

In [0]:
size_test_sample = 100 # 50 [10] zufällige Indexwerte für das Test-Set
index = np.random.randint(n_test, size = size_test_sample)
test_image_sample = gray_test_image[index]
test_labels_sample = test_labels[index]

In [0]:
labels_sample = labels_sample.reshape([size_image_sample])
test_labels_sample = test_labels_sample.reshape([size_test_sample])

# Setting up K and calculate distances

In [0]:
# setting up the k value 
k = 2 
#k = 4
#k = 8 
# calculating the distance between each vector in dependency of k and save it in an arra
distances = scipy.spatial.distance.cdist(image_sample, test_image_sample, metric='euclid')
# calculates the next k points around the vector
index_nearest = np.argpartition(distances, k, axis = 0)[:k] 

In [72]:
# saves the neighbors of the vector in dependency of k
knn_neighbors = labels_sample[index_nearest] 
# show the neighbors
print(knn_neighbors)

[[4 0 0 8 0 8 2 2 4 5 2 3 1 5 4 1 3 3 6 4 4 4 4 4 2 7 2 4 4 5 2 7 0 6 7 4
  8 8 4 4 9 6 8 4 8 8 4 8 2 0 1 6 2 6 7 8 2 4 5 7 2 2 4 2 2 6 6 3 0 5 0 0
  2 7 0 8 2 0 3 6 7 4 8 8 6 2 8 0 8 2 2 2 8 5 7 4 7 2 4 2]
 [7 0 0 9 4 2 4 3 4 6 2 6 0 3 2 6 7 6 4 0 0 4 7 2 2 5 4 5 2 4 2 4 2 4 5 0
  0 0 3 2 1 6 2 4 4 0 4 8 4 8 1 6 4 4 3 0 2 4 8 4 6 8 7 6 7 2 6 3 5 3 2 8
  4 9 3 2 4 4 4 9 2 2 8 0 4 4 0 5 0 2 4 0 1 4 7 2 2 4 0 0]]


In [73]:
# is looking for the most frequent value in the array/vector
knn_result = stats.mode(knn_neighbors)[0]
knn_result = knn_result[0]
# shows the most frequent value
knn_result

array([4, 0, 0, 8, 0, 2, 2, 2, 4, 5, 2, 3, 0, 3, 2, 1, 3, 3, 4, 0, 0, 4,
       4, 2, 2, 5, 2, 4, 2, 4, 2, 4, 0, 4, 5, 0, 0, 0, 3, 2, 1, 6, 2, 4,
       4, 0, 4, 8, 2, 0, 1, 6, 2, 4, 3, 0, 2, 4, 5, 4, 2, 2, 4, 2, 2, 2,
       6, 3, 0, 3, 0, 0, 2, 7, 0, 2, 2, 0, 3, 6, 2, 2, 8, 0, 4, 2, 0, 0,
       0, 2, 2, 0, 1, 4, 7, 2, 2, 2, 0, 0], dtype=uint8)

# Calculating error rate

In [0]:
errors = 0
# compares the two arrays and is looking for matches of the values? Right = true / Wrong = false
error_rate_matrix = (test_labels_sample == knn_result)
for b in error_rate_matrix:
    if b == False:
        errors += 1


In [78]:
np.unique(test_labels_sample, return_counts=True)

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=uint8),
 array([ 5, 11,  5, 14, 12,  9, 13, 13,  8, 10]))

In [79]:
# Calculation of the error rate
_rate = (errors / size_test_sample) * 100. 
print("Global error rate for k = " +str(k)+" "+str(_rate) + " %")

Global error rate for k = 2 86.0 %


Global error rate for k = 2 86.0 %


Global error rate for k = 4 78.0 %


Global error rate for k = 8 81.0 %